# Sistemas Lineares - Métodos Iterativos

Podemos manipular $Ax = b$ para uma equação do tipo ponto fixo: $x = \varphi(x)$.

Isso gera o método
$$ x^{k+1} = \varphi(x^k). $$

A pergunta é como escolher $\varphi$.

De uma maneira geral, vamos separar a matriz $A$ em duas, $A = M + N$, onde $M$ deve ser não-singular.
Isso irá garantir que a manipulação
$$ Ax = b \\
\Rightarrow Mx + Nx = b \\
\Rightarrow Mx = b - Nx \\
\Rightarrow x = M^{-1}b - M^{-1}Nx = g + Cx.
$$

Definimos $\varphi(x) = Cx + g$.

Note que

$$ N = A - M ⇒ Nx = Ax - Mx $$
$$ \varphi(x) = g - M^{-1}(Ax - Mx)
= g - M^{-1}(Ax) + x $$

Que também pode ser visto como
$$ \varphi(x) = x - M^{-1}(Ax - b). $$

Isso vai ser útil em breve.

Note que se $M = A$, então $\varphi(x) = x - A^{-1}Ax + A^{-1}b = A^{-1}b$, isto é, a partir de qualquer $x$, 
$\varphi(x)$ é a solução do sistema.
Computacionalmente, no entanto, $M = A$ não resolve nossos problemas.

Então, quem é $M$?

## Gauss-Jacobi

Uma estratégia é olhar para a diagonal de $A$.
$$ A = D + N $$

Se a diagonal não tiver elementos nulos, então podemos fazer a manipulação
$$ Dx + Nx = b \Rightarrow x = D^{-1}(b - Nx) = g + Cx, $$

In [89]:
A = [10.0 2 1; 1 5 1; 2 3 10]
b = [7.0; -8.0; 6.0]

3-element Array{Float64,1}:
  7.0
 -8.0
  6.0

In [90]:
A

3x3 Array{Float64,2}:
 10.0  2.0   1.0
  1.0  5.0   1.0
  2.0  3.0  10.0

In [91]:
D = diag(A) # Note que D não é uma matriz

3-element Array{Float64,1}:
 10.0
  5.0
 10.0

In [93]:
N = tril(A, -1) + triu(A, 1) # N = A - D

3x3 Array{Float64,2}:
 0.0  2.0  1.0
 1.0  0.0  1.0
 2.0  3.0  0.0

In [94]:
g = b./D

3-element Array{Float64,1}:
  0.7
 -1.6
  0.6

In [95]:
C = -N./D

3x3 Array{Float64,2}:
 -0.0  -0.2  -0.1
 -0.2  -0.0  -0.2
 -0.2  -0.3  -0.0

In [96]:
x = g # x⁰ = 0 ⇒ x¹ = g

3-element Array{Float64,1}:
  0.7
 -1.6
  0.6

In [117]:
x = C*x + g # Running a few times

3-element Array{Float64,1}:
  1.0
 -2.0
  1.0

## Implementar Gauss-Jacobi

$$ x^{k+1} = x^k - D^{-1}(Ax^k - b) $$

In [119]:
function gauss_jacobi(A, b; tol=1e-6, maxiter = 100)
    D = diag(A)
    n = length(b)
    x = b./D
    r = A*x - b
    maxD = maximum(abs(D)) # Escalamento
    iter = 0
    while norm(r, Inf)/maxD > tol
        x = x - r./D # D^{-1}(Ax-b) = D^{-1}*r = r./D
        iter += 1
        if iter >= maxiter
            break
        end
        r = A*x - b
    end
    return x, iter
end

gauss_jacobi (generic function with 1 method)

In [120]:
x, k = gauss_jacobi(A, b)

([1.00000026749376,-1.99999965324576,1.00000038780064],13)

In [129]:
# Modo C/Fortran
function gauss_jacobi2(A, b; tol=1e-6, maxiter = 100)
    n = length(b)
    x = zeros(n)
    xnew = zeros(n)
    for i = 1:n
        xnew[i] = b[i]/A[i,i]
    end
    # Esquece o escalamento por enquanto
    iter = 0
    while norm(xnew - x, Inf) > tol
        for i = 1:n
            x[i] = xnew[i]
        end
        for i = 1:n
            xnew[i] = b[i]
            for j = 1:n
                if i == j
                    continue # Pare o loop e vá para o próximo j
                end
                xnew[i] -= A[i,j] * x[j]
            end
            xnew[i] = xnew[i]/A[i,i]
        end
        iter += 1
        if iter >= maxiter
            break
        end
    end
    return xnew, iter
end

gauss_jacobi2 (generic function with 1 method)

In [128]:
gauss_jacobi2(A, b)

([0.9999998918690881,-2.00000013105888,0.9999998424749759],14)

### Complexidade

Cada loop faz $r./D$ que são $n$ divisões, $x - r./D$, que são $n$ somas e $r = Ax - b$
que são $n^2$ multiplicações e $n(n-1) + n = n^2$ somas. Ignorando o custo da $\Vert\cdots\Vert_{\infty}$,
temos que cada iteração do método de Gauss-Jacobi custa $2n^2 + 2n$ operações.

Fizemos 13 iterações, e nesse caso $n=3$, então esse método foi mais caro que LU. Vale a pena?

In [87]:
gauss_jacobi(A, b, tol=1e-2)

([0.9994000000000001,-1.9888000000000001,0.9984000000000001],4)

In [131]:
A = rand(100,100) + 100*eye(100)
b = A*ones(100)
x, k = gauss_jacobi(A, b)

([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],20)

## Gauss-Seidel

Olhe por extenso para o Gauss-Jacobi $3\times 3$:

\begin{align*}
x_1^{k+1} & = \frac{1}{a_{11}} (b_1 - a_{12} x_2^k - a_{13} x_3^k) \\
x_2^{k+1} & = \frac{1}{a_{22}} (b_2 - a_{21} x_1^k - a_{23} x_3^k) \\
x_3^{k+1} & = \frac{1}{a_{33}} (b_3 - a_{31} x_1^k - a_{32} x_2^k)
\end{align*}

Ao calcular $x_2^{k+1}$ já temos $x_1^{k+1}$ atualizado. Uma possível atualização seria fazer

\begin{align*}
x_1^{k+1} & = \frac{1}{a_{11}} (b_1 - a_{12} x_2^k - a_{13} x_3^k) \\
x_2^{k+1} & = \frac{1}{a_{22}} (b_2 - a_{21} x_1^{k+1} - a_{23} x_3^k) \\
x_3^{k+1} & = \frac{1}{a_{33}} (b_3 - a_{31} x_1^{k+1} - a_{32} x_2^{k+1})
\end{align*}

Essa escolha equivale a $M = L$, a triangular inferior de $A$ (a diagonal **não** é unitária).

In [132]:
# Modo C/Fortran
function gauss_seidel2(A, b; tol=1e-6, maxiter = 100)
    n = length(b)
    x = zeros(n)
    xnew = zeros(n)
    for i = 1:n
        xnew[i] = b[i]/A[i,i]
    end
    # Esquece o escalamento por enquanto
    iter = 0
    while norm(xnew - x, Inf) > tol
        for i = 1:n
            x[i] = xnew[i]
        end
        for i = 1:n
            xnew[i] = b[i]
            for j = 1:i-1
                xnew[i] -= A[i,j] * xnew[j]
            end
            for j = i+1:n
                xnew[i] -= A[i,j] * x[j]
            end
            xnew[i] = xnew[i]/A[i,i]
        end
        iter += 1
        if iter >= maxiter
            break
        end
    end
    return xnew, iter
end

gauss_seidel2 (generic function with 1 method)

In [133]:
gauss_jacobi2(A, b)

([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],21)

In [134]:
gauss_seidel2(A, b)

([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],8)

In [140]:
# Modo C/Fortran
function gauss_seidel3(A, b; tol=1e-6, maxiter = 100)
    n = length(b)
    x = zeros(n)
    # Esquece o escalamento por enquanto
    iter = 0
    errox = 1.0
    while errox > tol
        errox = 0.0
        for i = 1:n
            xi = x[i]
            s = b[i]
            for j = 1:i-1
                s -= A[i,j] * x[j]
            end
            for j = i+1:n
                s -= A[i,j] * x[j]
            end
            x[i] = s/A[i,i]
            errox = max(errox, abs(x[i] - xi))
        end
        iter += 1
        if iter >= maxiter
            break
        end
    end
    return x, iter
end

gauss_seidel3 (generic function with 1 method)

In [141]:
gauss_seidel3(A, b)

([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],8)

In [142]:
function tri_inf(L, b; tol=1e-12)
    n = length(b)
    c = zeros(n)
    for i = 1:n
        if abs(L[i,i]) < tol
            error("Lᵢᵢ ≈ 0.0")
        end
        c[i] = b[i]
        for j = i-1:-1:1
            c[i] -= L[i,j]*c[j]
        end
        c[i] = c[i]/L[i,i]
    end
    return c
end

tri_inf (generic function with 1 method)

In [143]:
function gauss_seidel(A, b; tol=1e-6, maxiter = 100)
    n = length(b)
    x = tri_inf(A, b)
    r = A*x - b
    maxD = maximum(abs(D)) # Escalamento
    iter = 0
    while norm(r, Inf)/maxD > tol
        x = x - tri_inf(A, r) # D^{-1}(Ax-b) = D^{-1}*r = r./D
        iter += 1
        if iter >= maxiter
            break
        end
        r = A*x - b
    end
    return x, iter
end

gauss_seidel (generic function with 1 method)

In [144]:
gauss_seidel(A, b)

([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],7)

## Convergência

- Critério das linhas
- Critério de Sassenfeld
- Raio espectral

In [154]:
A = [4.0 -1; 2 3]
b = A*[1.0; 2.0]

2-element Array{Float64,1}:
 2.0
 8.0

In [155]:
gauss_jacobi(A, b)

([0.9999997023129096,2.0000003969161204],16)

In [156]:
A = [2.0 3.0; 4.0 -1]
b = A*[1.0; 2.0]

2-element Array{Float64,1}:
 8.0
 2.0

In [157]:
gauss_jacobi(A, b)

([2.424843832394292e39,-3.233125109859056e39],100)

### Comparação

- Esparsidade
- Precisão